# Running Monte Carlo Transport Independently

This tutorial demonstrates how to run the Monte Carlo transport loop directly using `Simulation.from_config` without running full TARDIS iterations. This approach gives you direct control over the Monte Carlo transport process.

In [1]:
import os
from pathlib import Path

import astropy.units as u
import numpy as np

from tardis.simulation import Simulation
from tardis.io.configuration.config_reader import Configuration
from tardis.io.atom_data import AtomData
from tardis.transport.montecarlo.estimators.radfield_mc_estimators import (
    initialize_estimator_statistics,
)
from tardis.transport.montecarlo.montecarlo_main_loop import montecarlo_main_loop
from tardis.transport.montecarlo.packets.packet_trackers import (
    generate_rpacket_last_interaction_tracker_list,
    generate_rpacket_tracker_list,
)

/home/runner/work/tardis/tardis/tardis/__init__.py:17: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

Initializing tabulator and plotly panel extensions for widgets to work


In [2]:
!wget -q -nc https://raw.githubusercontent.com/tardis-sn/tardis/master/docs/tardis_example.yml

In [3]:
# User-configurable variables
CONFIG_FILE_NAME = "tardis_example.yml"
NUMBER_OF_PACKETS = 10000
NUMBER_OF_VPACKETS = 0  # Set to 0 to disable virtual packets
ITERATION_NUMBER = 1
SHOW_PROGRESS_BARS = True
TOTAL_ITERATIONS = 1
ENABLE_RPACKET_TRACKING = True  # True: full tracking, False: last interaction only

In [4]:
# Setup simulation state from config
config_file = Path(CONFIG_FILE_NAME)
if not config_file.exists():
    raise FileNotFoundError(f"Configuration file {CONFIG_FILE_NAME} not found")

config = Configuration.from_yaml(str(config_file))
atom_data = AtomData.from_hdf("kurucz_cd23_chianti_H_He_latest.h5")
sim = Simulation.from_config(config, atom_data=atom_data)

print("Simulation created successfully!")

Number of density points larger than number of shells. Assuming inner point irrelevant


model_isotope_time_0 is not set in the configuration. Isotopic mass fractions will not be decayed and is assumed to be correct for the time_explosion. THIS IS NOT RECOMMENDED!


Simulation created successfully!


In [5]:
# Initialize opacity and macro atom states manually
sim.opacity_state = sim.opacity.legacy_solve(sim.plasma)

if sim.macro_atom is not None:
    sim.macro_atom_state = sim.macro_atom.solve(
        sim.plasma.j_blues,
        sim.plasma.atomic_data,
        sim.opacity_state.tau_sobolev,
        sim.plasma.stimulated_emission_factor,
        sim.opacity_state.beta_sobolev,
    )
else:
    sim.macro_atom_state = None

print("Opacity and macro atom states initialized!")

Opacity and macro atom states initialized!


In [6]:
# Extract states from simulation
geometry_state = sim.simulation_state.geometry
opacity_state = sim.opacity_state
montecarlo_configuration = sim.transport.montecarlo_configuration
time_explosion = sim.simulation_state.time_explosion.to(u.s).value
spectrum_frequency_grid = sim.transport.spectrum_frequency_grid.to(u.Hz).value
packet_source = sim.transport.packet_source

# Initialize estimators
tau_sobolev_shape = opacity_state.tau_sobolev.shape
gamma_shape = (0, geometry_state.no_of_shells)
estimators = initialize_estimator_statistics(tau_sobolev_shape, gamma_shape)

# Convert to numba-compatible versions
geometry_state_numba = geometry_state.to_numba()
line_interaction_type = montecarlo_configuration.LINE_INTERACTION_TYPE
opacity_state_numba = opacity_state.to_numba(sim.macro_atom_state, line_interaction_type)

print("Monte Carlo states prepared!")

Monte Carlo states prepared!


In [7]:
# Create packet collection
seed_offset = montecarlo_configuration.PACKET_SEEDS
packet_collection = packet_source.create_packets(NUMBER_OF_PACKETS, seed_offset)

# Setup packet tracking
if ENABLE_RPACKET_TRACKING:
    rpacket_trackers = generate_rpacket_tracker_list(
        NUMBER_OF_PACKETS,
        montecarlo_configuration.INITIAL_TRACKING_ARRAY_LENGTH,
    )
    print("Using full RPacket tracking")
else:
    rpacket_trackers = generate_rpacket_last_interaction_tracker_list(
        NUMBER_OF_PACKETS
    )
    print("Using last interaction tracking only")


Using full RPacket tracking


In [8]:
# Run the Monte Carlo main loop
v_packets_energy_hist, last_interaction_tracker, vpacket_tracker = (
    montecarlo_main_loop(
        packet_collection,
        geometry_state_numba,
        time_explosion,
        opacity_state_numba,
        montecarlo_configuration,
        estimators,
        spectrum_frequency_grid,
        rpacket_trackers,
        NUMBER_OF_VPACKETS,
        SHOW_PROGRESS_BARS
    )
)

print("Monte Carlo transport completed successfully!")

/home/runner/work/tardis/tardis/tardis/transport/montecarlo/montecarlo_main_loop.py:150: NumbaTypeSafetyWarning:

unsafe cast from uint64 to int64. Precision may be lost.



Monte Carlo transport completed successfully!


In [9]:
# Inspect results
print("Monte Carlo Results:")
print(f"V-packet energy histogram shape: {v_packets_energy_hist.shape}")
print(f"Last interaction tracker type: {type(last_interaction_tracker)}")
if ENABLE_RPACKET_TRACKING:
    print(f"V-packet tracker available: {type(vpacket_tracker)}")

# Results are now available in the notebook for further analysis:
# - v_packets_energy_hist: energy histogram of virtual packets
# - last_interaction_tracker: final interaction data for all packets  
# - vpacket_tracker: virtual packet collection (if enabled)
# - sim: original simulation object

Monte Carlo Results:
V-packet energy histogram shape: (10001,)
Last interaction tracker type: <class 'numba.experimental.jitclass.boxing.LastInteractionTracker'>
V-packet tracker available: <class 'numba.experimental.jitclass.boxing.VPacketCollection'>


## Usage Instructions

### Configuration Options:
- **NUMBER_OF_PACKETS**: Number of packets to simulate (default: 10000)
- **NUMBER_OF_VPACKETS**: Number of virtual packets per interaction (0 = disabled)
- **ENABLE_RPACKET_TRACKING**: True for full tracking, False for last interaction only

### For Development and Debugging:
1. Set `NUMBA_DISABLE_JIT=1` in your environment variables to disable Numba JIT compilation
2. Set breakpoints in cell 8 (the Monte Carlo main loop call) 
3. Run the notebook in debug mode to step through the transport process

### Key Advantages Over Full TARDIS Run:
- **Direct control**: Access Monte Carlo transport without full simulation iterations
- **Manual state initialization**: All states explicitly prepared for transparency
- **Clean separation**: Setup and execution are clearly separated
- **Educational**: Perfect for understanding Monte Carlo transport physics step-by-step

# Running Monte Carlo Transport Loop

This tutorial demonstrates how to run the `montecarlo_main_loop` function directly with the TARDIS quickstart configuration.

In [10]:
import numpy as np
import astropy.units as u
from pathlib import Path

from tardis import run_tardis
from tardis.transport.montecarlo.montecarlo_main_loop import montecarlo_main_loop
from tardis.transport.montecarlo.packets.packet_collections import PacketCollection
from tardis.transport.montecarlo.packets.packet_trackers import (
    generate_rpacket_tracker_list,
    generate_rpacket_last_interaction_tracker_list
)

In [11]:
print("Monte Carlo Results:")
print(f"V-packet energy histogram shape: {v_packets_energy_hist.shape}")
print(f"Last interaction tracker type: {type(last_interaction_tracker)}")
if ENABLE_RPACKET_TRACKING:
    print(f"V-packet tracker available: {type(vpacket_tracker)}")

Monte Carlo Results:
V-packet energy histogram shape: (10001,)
Last interaction tracker type: <class 'numba.experimental.jitclass.boxing.LastInteractionTracker'>
V-packet tracker available: <class 'numba.experimental.jitclass.boxing.VPacketCollection'>


In [12]:
sim = run_tardis(CONFIG_FILE_NAME, show_progress_bars=SHOW_PROGRESS_BARS)

Auto-detected Sphinx build environment


Auto-detected Sphinx build environment


Embedding the final state for Jupyter environments


In [13]:
from tardis.transport.montecarlo.estimators.radfield_mc_estimators import initialize_estimator_statistics

# Get the geometry from simulation_state
geometry_state = sim.simulation_state.geometry
# Get the opacity state from the simulation (not from transport.transport_state)
opacity_state = sim.opacity_state
montecarlo_configuration = sim.transport.montecarlo_configuration
time_explosion = sim.simulation_state.time_explosion.to(u.s).value

# Initialize estimators using the same function used in TARDIS
tau_sobolev_shape = opacity_state.tau_sobolev.shape
# For continuum estimators, check if we have continuum processes
if hasattr(opacity_state, 'continuum_state') and opacity_state.continuum_state is not None:
    gamma_shape = (opacity_state.continuum_state.photo_ion_idx.max() + 1, geometry_state.no_of_shells)
else:
    gamma_shape = (0, geometry_state.no_of_shells)

estimators = initialize_estimator_statistics(tau_sobolev_shape, gamma_shape)
spectrum_frequency_grid = sim.transport.spectrum_frequency_grid.to(u.Hz).value
packet_source = sim.transport.packet_source

# Convert objects to numba-compatible versions
geometry_state_numba = geometry_state.to_numba()
macro_atom_state = sim.macro_atom_state if hasattr(sim, 'macro_atom_state') else None
line_interaction_type = montecarlo_configuration.LINE_INTERACTION_TYPE
opacity_state_numba = opacity_state.to_numba(macro_atom_state, line_interaction_type)

In [14]:
seed_offset = montecarlo_configuration.PACKET_SEEDS
packet_collection = packet_source.create_packets(NUMBER_OF_PACKETS, seed_offset)

if ENABLE_RPACKET_TRACKING:
    rpacket_trackers = generate_rpacket_tracker_list(
        NUMBER_OF_PACKETS,
        montecarlo_configuration.INITIAL_TRACKING_ARRAY_LENGTH,
    )
else:
    rpacket_trackers = generate_rpacket_last_interaction_tracker_list(
        NUMBER_OF_PACKETS
    )

In [15]:
%%timeit
# Run the Monte Carlo main loop
# Reinitialize estimators (they get modified by the main loop)
estimators = initialize_estimator_statistics(tau_sobolev_shape, gamma_shape)

# Run the Monte Carlo main loop
v_packets_energy_hist, last_interaction_tracker, vpacket_tracker = montecarlo_main_loop(
    packet_collection,
    geometry_state_numba,
    time_explosion,
    opacity_state_numba,
    montecarlo_configuration,
    estimators,
    spectrum_frequency_grid,
    rpacket_trackers,
    NUMBER_OF_VPACKETS,
    SHOW_PROGRESS_BARS
)



166 ms ± 2.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Tracking Options Explained (TARDIS Implementation)

### Important: LastInteractionTracker is Always Available
The `montecarlo_main_loop` **always** returns a `LastInteractionTracker` object containing the final interaction data for all packets, regardless of the tracking option chosen.

### Option 1: Full RPacket Tracking (`ENABLE_RPACKET_TRACKING = True`)
- **Equivalent to**: `config.montecarlo.tracking.track_rpacket = True` in TARDIS config
- **Function used**: `generate_rpacket_tracker_list()`
- **RPacket Data**: Records position (r), frequency (nu), direction (mu), energy, shell_id, and interaction_type at **each step** of every packet's journey
- **LastInteractionTracker**: Also populated with final interaction data
- **Use case**: Detailed analysis of packet trajectories, debugging transport physics, studying packet paths
- **Memory**: Higher memory usage as it stores the complete history for each packet

### Option 2: Last Interaction Only (`ENABLE_RPACKET_TRACKING = False`)  
- **Equivalent to**: `config.montecarlo.tracking.track_rpacket = False` in TARDIS config (default)
- **Function used**: `generate_rpacket_last_interaction_tracker_list()`
- **RPacket Data**: Uses `RPacketLastInteractionTracker` objects that store only final interaction per packet
- **LastInteractionTracker**: Contains aggregated final interaction data for all packets
- **Use case**: Statistical analysis of where packets finally interact, line formation regions
- **Memory**: Lower memory usage as it stores only final interaction data

### Key Differences:
- **Full tracking**: Each packet gets an `RPacketTracker` that records every interaction step
- **Last interaction only**: Each packet gets an `RPacketLastInteractionTracker` that records only the final interaction

**To switch between options:** Change `ENABLE_RPACKET_TRACKING` in cell 6 and re-run cells 6-8.

## Refactoring Recommendations for Monte Carlo Subsystem

Based on this tutorial, here are key areas that could benefit from refactoring for greater clarity:

### 1. **Inconsistent Object Access Patterns**
- **Current**: Mixed access through `sim.transport.transport_state`, `sim.opacity_state`, `sim.simulation_state`
- **Recommendation**: Create a unified interface or facade pattern that provides consistent access to all MC components

### 2. **Complex State Conversion**
- **Current**: Manual conversion to numba-compatible objects with required parameters
- **Recommendation**: Create helper functions or factory methods that handle numba conversions automatically

### 3. **Estimator Initialization Complexity** 
- **Current**: Manual shape calculation and estimator initialization
- **Recommendation**: Create an `EstimatorFactory` that handles all the shape calculations and initialization logic

### 4. **Packet Creation Indirection**
- **Current**: Multiple steps to create packets (source → create_packets → collection)
- **Recommendation**: Simplified packet creation API that reduces the number of steps

### 5. **Configuration Parameter Inconsistency**
- **Current**: Some parameters exist in config, others have defaults, naming is inconsistent
- **Recommendation**: Standardize configuration parameter naming and provide clear defaults

### 6. **Type System Clarity**
- **Current**: Mix of Python objects, Numba objects, with unclear conversion points
- **Recommendation**: Clear separation and documentation of when objects need to be converted

### 7. **Function Parameter Count**
- **Current**: `montecarlo_main_loop` takes 12+ parameters
- **Recommendation**: Create structured parameter objects or context managers

### Proposed Refactored Interface

Instead of the current complex setup, a cleaner interface could look like:

```python
# Proposed simplified API
from tardis.transport.montecarlo import MonteCarloRunner

# Create runner with simulation
mc_runner = MonteCarloRunner.from_simulation(sim)

# Configure run parameters
mc_config = MonteCarloRunConfig(
    number_of_packets=1000,
    number_of_vpackets=10,
    iteration=1,
    show_progress_bars=True
)

# Run the main loop with simplified interface
results = mc_runner.run_main_loop(mc_config)

# Access results with clear structure
print(f"V-packets histogram: {results.vpacket_energy_histogram.shape}")
print(f"Interaction tracker: {results.last_interaction_tracker.count}")
```

This would encapsulate all the complexity of:
- State extraction and conversion
- Estimator initialization  
- Packet creation and tracking
- Numba object preparation